In [1]:
import sys
sys.path.append('../../../')

In [2]:
import os
import torch
import argparse
import pandas as pd
from lightsaber.model_lib.pt_sota_models import rnn
from lightsaber.data_utils import pt_dataset as ptd
import lightsaber.data_utils.utils as du
from sklearn.preprocessing import StandardScaler
import mlflow

pysurvival not installed... survival models wont work


In [3]:
import logging
log = logging.getLogger()

In [4]:
checkpoint_file='epoch=1.ckpt'

In [5]:
from lightsaber.data_utils import pt_dataset as ptd


In [6]:
data_dir = '/Users/rachitachandra/Desktop/dpm2/lightsaber/examples/in_hosptial_mortality/data'
expt_conf = du.yaml.load(open('..//archive/ohdsi_ihm_expt_config.yml').read().format(DATA_DIR=data_dir),
                         Loader=du._Loader)

In [7]:
a = pd.read_csv(expt_conf['test']['feat_file'])

/Users/rachitachandra/anaconda3/envs/dpm3601/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
@ptd.functoolz.curry
def filter_fillna(data, target, fill_value=0., time_order_col=None):
    data = data.copy()
    
    idx_cols = data.index.names
    if time_order_col is not None:
        try:
            sort_cols = idx_cols + time_order_col
        except:
            sort_cols = idx_cols + [time_order_col]
    else:
        sort_cols = idx_cols
    
    data.update(data.reset_index()
               .sort_values(sort_cols)
               .groupby(idx_cols[0])
               .ffill())
    
    data.fillna(fill_value, inplace=True)
        
    return data, target


@ptd.functoolz.curry
def filter_preprocessor(data, target, cols=None, preprocessor=None, refit=False):
    if preprocessor is not None:
        all_columns = data.columns
        index = data.index

        # Extracting the columns to fit
        if cols is None:
            cols = all_columns
        _oCols = all_columns.difference(cols)
        xData = data[cols]
    
        # If fit required fitting it
        if refit:
            preprocessor.fit(xData)
            log.info(f'Fitting pre-proc: {preprocessor}')
  
        # Transforming data to be transformed
        try:
            xData = preprocessor.transform(xData)
        except NotFittedError:
            raise Exception(f"{preprocessor} not fitted. pass fitted preprocessor or set refit=True")
        xData = pd.DataFrame(columns=cols, data=xData, index=index)
        
        # Merging other columns if required
        if not _oCols.empty:
            tmp = pd.DataFrame(data=data[_oCols].values, 
                               columns=_oCols,
                               index=index)
            xData = pd.concat((tmp, xData), axis=1)
        
        # Re-ordering the columns to original order
        data = xData[all_columns]
    return data, target

In [9]:
preprocessor = StandardScaler()
train_filter = [filter_preprocessor(cols=expt_conf['numerical'], 
                                    preprocessor=preprocessor,
                                    refit=True),
                filter_fillna(fill_value=expt_conf['normal_values'],
                              time_order_col=expt_conf['time_order_col'])
                ]
transform = ptd.transform_drop_cols(cols_to_drop=expt_conf['time_order_col'])

In [10]:
train_dataset = ptd.BaseDataset(tgt_file=expt_conf['train']['tgt_file'],
                                feat_file=expt_conf['train']['feat_file'],
                                idx_col=expt_conf['idx_cols'],
                                tgt_col=expt_conf['tgt_col'],
                                feat_columns=expt_conf['feat_cols'],
                                time_order_col=expt_conf['time_order_col'],
                                category_map=expt_conf['category_map'],
                                transform=transform,
                                filter=train_filter,
                               )

/Users/rachitachandra/anaconda3/envs/dpm3601/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
input_dim, target_dim = train_dataset.shape
output_dim = 2

In [12]:
hparams = argparse.Namespace(#gpus=None,
                             lr=0.01,
                             max_epochs=2,
                             batch_size=32,
                             hidden_dim=16,
                             rnn_class='LSTM',
                             n_layers=1,
                             dropout=0.1,
                             recurrent_dropout=0.1,
                             bidirectional=False,
                             )


base_model = rnn.RNNClassifier(input_dim, output_dim, 
                               hidden_dim=hparams.hidden_dim,
                               rnn_class=hparams.rnn_class,
                               n_layers=hparams.n_layers,
                               dropout=hparams.dropout,
                               recurrent_dropout=hparams.recurrent_dropout,
                               bidirectional=hparams.bidirectional
                              )


/Users/rachitachandra/anaconda3/envs/dpm3601/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [13]:
base_model

RNNClassifier(
  (rnn): LSTM(38, 16, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.1, inplace=False)
  (logit): Linear(in_features=16, out_features=2, bias=False)
)

In [14]:
from torch import nn
import torch as T
from lightsaber.trainers import pt_trainer as ptr


# Creating the wrapped model

weight_labels = train_dataset.target.iloc[:, 0].value_counts()
weight_labels = (weight_labels.max() / ((weight_labels + 0.0000001) ** (1)))
weight_labels.sort_index(inplace=True)
weights = T.FloatTensor(weight_labels.values).to(train_dataset.device)
print(weights)
criterion = nn.CrossEntropyLoss(weight=weights)
test_dataset = pd.read_csv(expt_conf['test']['feat_file'])
wrapped_model = ptr.PyModel(hparams, base_model,
                            train_dataset=train_dataset,
                            val_dataset=None, # None
                            test_dataset=test_dataset, # test_dataset
                            #optimizer=optimizer,
                            loss_func=criterion,
                            #scheduler=scheduler,
                            collate_fn=ptd.collate_fn
                            )

tensor([ 1.0000, 21.4786])


/Users/rachitachandra/anaconda3/envs/dpm3601/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
checkpoint_file='/Users/rachitachandra/epoch=1.ckpt'    
checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))
wrapped_model.load_state_dict(checkpoint['state_dict'])

#base_model.load_state_dict(checkpoint['model'])


<All keys matched successfully>

In [16]:
wrapped_model.eval()

PyModel(
  (model): RNNClassifier(
    (rnn): LSTM(38, 16, batch_first=True, dropout=0.1)
    (dropout): Dropout(p=0.1, inplace=False)
    (logit): Linear(in_features=16, out_features=2, bias=False)
  )
  (loss_func): CrossEntropyLoss()
  (out_transform): Softmax(dim=1)
)

In [17]:
test_dataset = ptd.BaseDataset(tgt_file=expt_conf['test']['tgt_file'],
                                feat_file=expt_conf['test']['feat_file'],
                                idx_col=expt_conf['idx_cols'],
                                tgt_col=expt_conf['tgt_col'],
                                feat_columns=expt_conf['feat_cols'],
                                time_order_col=expt_conf['time_order_col'],
                                category_map=expt_conf['category_map'],
                                transform=transform,
                                filter=train_filter,
                               )

/Users/rachitachandra/anaconda3/envs/dpm3601/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
test_dataset.data

,seqnum,hours,Body height,Body temperature,Body weight,Diastolic blood pressure,Heart rate,Inhaled oxygen concentration,Mean blood pressure,Oxygen saturation in Arterial blood,...,Glasgow coma scale_L11,Glasgow coma scale_L12,Glasgow coma scale_L13,Glasgow coma scale_L14,Glasgow coma scale_L15,Glasgow coma score verbal_No response,Glasgow coma score verbal_Incomprensible sounds,Glasgow coma score verbal_Inappropriate words,Glasgow coma score verbal_Confused,Glasgow coma score verbal_Oriented
stayId,,,,,,,,,,,,,,,,,,,,,
392803055_397734998,0,0.969722,0.075929,-0.212329,-0.030287,-0.005466,-0.004780,-0.309293,-0.024341,0.031314,...,0,0,0,0,1,0,0,0,0,1
392803055_397734998,1,0.986389,0.075929,-0.068937,-0.030287,-0.005466,-0.004780,2.027032,-0.024341,0.031314,...,0,0,0,0,1,1,0,0,0,0
392803055_397734998,2,1.03639,0.075929,-0.068937,-0.030287,0.343020,0.853797,-0.309293,2.401530,0.031314,...,0,0,0,0,1,0,0,0,0,1
392803055_397734998,3,1.05306,0.075929,-0.068937,-0.030287,-0.005466,-0.004780,-0.309293,-0.024341,0.135146,...,0,0,0,0,1,0,0,0,0,1
392803055_397734998,4,1.10306,0.075929,-0.068937,-0.422787,-0.005466,-0.004780,-0.309293,-0.024341,0.031314,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392814937_397802481,0,9.475,0.075929,-0.068937,-0.222063,-0.005466,-0.004780,-0.309293,-0.024341,0.031314,...,0,0,0,0,1,0,0,0,0,1
392817845_642740892,0,0.683333,0.075929,-0.068937,-2.006475,-0.005466,-0.004780,-0.309293,-0.024341,0.031314,...,0,0,0,0,1,0,0,0,0,1
392800214_397758926,0,0.402778,0.075929,-0.068937,0.315243,-0.005466,-0.004780,-0.309293,-0.024341,0.031314,...,0,0,0,0,1,0,0,0,0,1


In [19]:
from torch.utils.data import DataLoader

In [20]:
DataLoader(test_dataset)

In [24]:
dataloader = DataLoader(test_dataset, collate_fn=None, pin_memory=False,
                                    batch_size=32,num_workers=4)

In [25]:
test_dataloader = wrapped_model.test_dataloader()

In [27]:
(test_pred_proba, test_yhat, 
             test_y, test_score) = wrapped_model.run(test_dataloader, 
                                                     overfit_pct= 0)

  0%|          | 0/32569 [00:00<?, ?it/s]


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/rachitachandra/anaconda3/envs/dpm3601/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2895, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/rachitachandra/anaconda3/envs/dpm3601/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/rachitachandra/a

In [71]:
wrapped_model.eval()
import torch as nn
import numpy as np
wrapped_model.predict_proba(nn.tensor([test_dataset.data.values.astype(np.float64)]),"cpu")

TypeError: new(): invalid data type 'str'

In [34]:
tracking_uri = "https://foh-ohdsi-dev-mlflow.bx.cloud9.ibm.com"

In [35]:
mlflow.set_tracking_uri(tracking_uri)

In [15]:
mlflow.search_runs('0','')

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,metrics.mae,metrics.r2,params.l1_ratio,params.alpha,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.log-model.history,tags.mlflow.source.git.commit
0,af48be0fc7d34406b830e25101e81b59,0,FINISHED,s3://mlflow-experiments/0/af48be0fc7d34406b830...,2021-05-14 23:02:39.752000+00:00,2021-05-14 23:04:16.971000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
1,1f1c5b5d7b85462bbfe236bcf42bcb33,0,FAILED,s3://mlflow-experiments/0/1f1c5b5d7b85462bbfe2...,2021-05-14 22:43:57.190000+00:00,2021-05-14 22:43:59.830000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
2,fda292f2741e4cf5abf31268b5301231,0,FAILED,s3://mlflow-experiments/0/fda292f2741e4cf5abf3...,2021-05-14 22:41:01.884000+00:00,2021-05-14 22:41:04.632000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
3,367836965c564d2983102b7cc760cfe5,0,FAILED,s3://mlflow-experiments/0/367836965c564d298310...,2021-05-14 22:36:43.226000+00:00,2021-05-14 22:36:45.822000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
4,32ab33d1baed40b685191894abb30a3d,0,FAILED,s3://mlflow-experiments/0/32ab33d1baed40b68519...,2021-05-14 22:36:05.126000+00:00,2021-05-14 22:36:07.747000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
5,44247aa7c6e746389091d9f54e389ae2,0,FAILED,s3://mlflow-experiments/0/44247aa7c6e746389091...,2021-05-14 22:24:47.780000+00:00,2021-05-14 22:24:50.760000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
6,22eb4b68ff6d43d7bfd40832c9475a3e,0,FAILED,s3://mlflow-experiments/0/22eb4b68ff6d43d7bfd4...,2021-05-14 22:17:58.375000+00:00,2021-05-14 22:18:01.244000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
7,2694984805b8458b8632f298d3fb767e,0,FAILED,s3://mlflow-experiments/0/2694984805b8458b8632...,2021-05-14 22:17:06.161000+00:00,2021-05-14 22:17:09.360000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
8,ae2a03f9ddec4c2a9d19899a96018b72,0,FAILED,s3://mlflow-experiments/0/ae2a03f9ddec4c2a9d19...,2021-05-14 22:16:00.781000+00:00,2021-05-14 22:16:01.676000+00:00,NaN,NaN,NaN,None,None,LOCAL,rachitachandra,/Users/rachitachandra/anaconda3/envs/dpm3601/l...,None,None
9,48eafe70036d4f7386c3dc87f7049324,0,FINISHED,s3://mlflow-experiments/0/48eafe70036d4f7386c3...,2021-05-14 15:18:44.716000+00:00,2021-05-14 15:19:13.673000+00:00,0.793164,0.627195,0.108626,0.5,0.5,LOCAL,nelsonbore,sample_train.py,"[{""run_id"": ""48eafe70036d4f7386c3dc87f7049324""...",6a7851fff3c8ee8e4f1a7705cbf3d96309fdefdf


In [84]:
!pip install boto3

     |████████████████████████████████| 98 kB 599 kB/s eta 0:00:01
     |████████████████████████████████| 7.5 MB 297 kB/s eta 0:00:01
     |████████████████████████████████| 79 kB 620 kB/s eta 0:00:01
  Created wheel for boto3: filename=boto3-1.17.73-py2.py3-none-any.whl size=128919 sha256=f4260cdab2afa1f45d179a0dca87c2a49ed456683cda4a17fcb3e38d65196f52
  Stored in directory: /Users/rachitachandra/Library/Caches/pip/wheels/4e/6c/81/6da8693e26b14915229799441d3c82a910dc6d9d37d6895866
Successfully built boto3


In [36]:
#os.environ['AWS_ACCESS_KEY_ID']='minioRoot'
os.environ['MLFLOW_S3_ENDPOINT_URL']='https://foh-ohdsi-dev-minio.bx.cloud9.ibm.com'
with mlflow.start_run() as run:
    mlflow.pytorch.log_model(base_model,'dpm360')

In [21]:
mlflow.end_run()

In [37]:
model_uri = "runs:/{}/pytorch-model".format(run.info.run_id)
mv = mlflow.register_model(model_uri, "DPM360Model")



Successfully registered model 'DPM360Model'.
2021/05/14 20:11:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: DPM360Model, version 1
Created version '1' of model 'DPM360Model'.
